In [1]:
#sys.path.append("C:/Users/Administrator/Desktop/Desktop/ADE-TFT")
import pandas as pd
import os
import warnings
warnings.filterwarnings("ignore")  # avoid printing out absolute paths
import copy
from pathlib import Path
import warnings
import numpy as np
import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping, LearningRateMonitor
from pytorch_lightning.loggers import TensorBoardLogger
import torch
from pytorch_forecasting import Baseline, TemporalFusionTransformer, TimeSeriesDataSet
from pytorch_forecasting.data import GroupNormalizer
from pytorch_forecasting.metrics import SMAPE, PoissonLoss, QuantileLoss
from pytorch_forecasting.models.temporal_fusion_transformer.tuning import optimize_hyperparameters
import tensorflow as tf
import tensorboard as tb


2023-05-10 10:32:25.906772: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-10 10:32:28.531275: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-05-10 10:32:28.531488: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-05-10 10:32:28.531504: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

In [2]:
def MyTFT(XXXX):

 csv_path = '/home/nouman/Downloads/arslan work/data.csv'
 data = pd.read_csv(csv_path)
 tf.io.gfile = tb.compat.tensorflow_stub.io.gfile
 data["id"] = "tourism volume"
 data["time_idx"] = pd.to_datetime(data["data"]).astype(np.int64)
 data["time_idx"] -= data["time_idx"].min()
 data["time_idx"] = (data.time_idx / 3600000000000) + 1
 data["time_idx"] = data["time_idx"].astype(int)
 data1=data
 data = data1[:41]
 max_prediction_length = 1
 max_encoder_length = int(XXXX[0])
 training_cutoff = data["time_idx"].max() - max_prediction_length
 test_data = data[lambda x: x.time_idx > x.time_idx.max() - max_encoder_length]
 training = TimeSeriesDataSet(
    data[lambda x: x.time_idx <= training_cutoff],
    time_idx="time_idx",
    target="tourism volume",
    group_ids=["id"],
    min_encoder_length=max_encoder_length // 2,  # keep encoder length long (as it is in the validation set)
    max_encoder_length=max_encoder_length,
    min_prediction_length=1,
    max_prediction_length=max_prediction_length,
    static_categoricals=["id"],
    time_varying_known_reals=["time_idx","Month","Year"],
    time_varying_unknown_categoricals=[],
    time_varying_unknown_reals=["tourism volume","Confirmed cases","number of posts","CNN values","Paris museum","Paris airport","Paris flight"
                                ],
    target_normalizer=GroupNormalizer(
        groups=["id"], transformation="softplus"
    ),
    add_relative_time_idx=True,
    add_target_scales=True,
    add_encoder_length=True,
 )
 validation = TimeSeriesDataSet.from_dataset(training, data, predict=True, stop_randomization=True)
 batch_size = int(XXXX[1])  # set this between 32 to 128
 train_dataloader = training.to_dataloader(train=True, batch_size=batch_size, num_workers=0)
 val_dataloader = validation.to_dataloader(train=False, batch_size=batch_size * 10, num_workers=0)
 pl.seed_everything(42)
 early_stop_callback = EarlyStopping(monitor="val_loss", min_delta=1e-4, patience=20, verbose=False, mode="min")
 lr_logger = LearningRateMonitor()  # log the learning rate
 logger = TensorBoardLogger("lightning_logs")  # logging results to a tensorboard
 trainer = pl.Trainer(
    max_epochs=10,#30
    gpus=0,
    weights_summary="top",
    gradient_clip_val=0.1,
    limit_train_batches=30,  # coment in for training, running valiation every 30 batches
    callbacks=[lr_logger, early_stop_callback],
    logger=logger,
 )
 tft = TemporalFusionTransformer.from_dataset(
    training,
    #learning_rate= 0.1,
    learning_rate= XXXX[4],
    hidden_size=int(XXXX[2]),
    attention_head_size=int(XXXX[5]),
    dropout=0.1,
    hidden_continuous_size=int(XXXX[3]),
    output_size=7,  # 7 quantiles by default
    loss=QuantileLoss(),
    log_interval=10,  # uncomment for learning rate finder and otherwise, e.g. to 10 for logging every 10 batches
    reduce_on_plateau_patience=4,
 )
 trainer.fit(
    tft,
    train_dataloaders=train_dataloader,
    val_dataloaders=val_dataloader,
 )
 raw_predictions, x = tft.predict(val_dataloader, mode="raw", return_x=True)
 tft.plot_prediction(x, raw_predictions, idx=0, add_loss_to_title=True)
 new_raw_predictions, new_x =  tft.predict(
    test_data,
    mode="raw",
    return_x=True
 )
 tft.plot_prediction(
    new_x,
    new_raw_predictions,
    idx=0,
    show_future_observed=False
 )
 result=[]
 len_data=len(data1)
 test_len=6
 test_y=[]
 for i in range(test_len):
    test_data=data1[len_data-max_encoder_length-(test_len-i)+1:len_data-(test_len-i)+1]
    new_raw_predictions, new_x =  tft.predict(
    test_data,
    mode="raw",
    return_x=True
    )
    a=new_raw_predictions['prediction']
    b=a.numpy()
    result.append(b[0,0,3])
 test_y1=data1["tourism volume"]
 test_y2=test_y1[len_data-test_len:len_data]
 test_y=[]
 for i in range(test_len):
    test_y.append(test_y2[[len_data+i-test_len]])
 result=np.array(result)
 test_y=np.array(test_y)
 MAPE1 = 0
 for i in range(test_len):
    MAPE1=MAPE1+np.abs(result[i]-test_y[i])/test_y[i]
 MAPE1 = MAPE1/test_len
 print(XXXX)
 return  MAPE1

In [3]:
XXXX= [128, 32, 64, 32, 0.01, 16]

In [4]:
import pandas as pd
df= pd.read_csv('data.csv')
df

,Date;Open;High;Low;Adj Close;Volume;capmrktcurusd;asseteodcompletiontime;PriceUSD;Close
0,17/09/2014;465.864014;468.174011;452.421997;45...
1,18/09/2014;456.859985;456.859985;413.104004;42...
2,19/09/2014;424.102997;427.834991;384.532013;39...
3,20/09/2014;394.673004;423.29599;389.882996;408...
4,21/09/2014;408.084991;412.425995;393.181;398.8...
...,...
2989,23/11/2022;16195.58887;16638.19336;16170.50293...
2990,24/11/2022;16611.63672;16771.47461;16501.76758...
2991,25/11/2022;16602.26953;16603.31641;16388.4043;...
2992,26/11/2022;null;null;null;null;null;6.27E+11;;...


In [5]:
df= df['Date;Open;High;Low;Adj Close;Volume;capmrktcurusd;asseteodcompletiontime;PriceUSD;Close'].str.split(';', expand=True)


In [6]:
df.head()

,0,1,2,3,4,5,6,7,8,9
0,17/09/2014,465.864014,468.174011,452.421997,457.334015,21056800,6060691069,1614336399,267.7976174,457.334015
1,18/09/2014,456.859985,456.859985,413.104004,424.440002,34483200,5671343317,1614336400,259.6136959,424.440002
2,19/09/2014,424.102997,427.834991,384.532013,394.79599,37919700,5234786700,1614336400,236.3808189,394.79599
3,20/09/2014,394.673004,423.29599,389.882996,408.903992,36863600,5464482173,1614336401,217.1934725,408.903992
4,21/09/2014,408.084991,412.425995,393.181,398.821014,26580100,5324715501,1614336402,234.3929975,398.821014


In [7]:
df.columns = (['Date', 'Open', 'High', 'Low', 'Adj Close', 'Volume', 'capmrktcurusd','asseteodcompletiontime', 'PriceUSD', 'Close'])

In [8]:
df.head()

,Date,Open,High,Low,Adj Close,Volume,capmrktcurusd,asseteodcompletiontime,PriceUSD,Close
0,17/09/2014,465.864014,468.174011,452.421997,457.334015,21056800,6060691069,1614336399,267.7976174,457.334015
1,18/09/2014,456.859985,456.859985,413.104004,424.440002,34483200,5671343317,1614336400,259.6136959,424.440002
2,19/09/2014,424.102997,427.834991,384.532013,394.79599,37919700,5234786700,1614336400,236.3808189,394.79599
3,20/09/2014,394.673004,423.29599,389.882996,408.903992,36863600,5464482173,1614336401,217.1934725,408.903992
4,21/09/2014,408.084991,412.425995,393.181,398.821014,26580100,5324715501,1614336402,234.3929975,398.821014


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2994 entries, 0 to 2993
Data columns (total 10 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Date                    2994 non-null   object
 1   Open                    2994 non-null   object
 2   High                    2994 non-null   object
 3   Low                     2994 non-null   object
 4   Adj Close               2994 non-null   object
 5   Volume                  2994 non-null   object
 6   capmrktcurusd           2994 non-null   object
 7   asseteodcompletiontime  2994 non-null   object
 8   PriceUSD                2994 non-null   object
 9   Close                   2994 non-null   object
dtypes: object(10)
memory usage: 234.0+ KB


In [10]:
df.nunique()

Date                      2994
Open                      2990
High                      2991
Low                       2993
Adj Close                 2991
Volume                    2994
capmrktcurusd             1883
asseteodcompletiontime    2183
PriceUSD                  2782
Close                     2991
dtype: int64

In [11]:
df.tail()

,Date,Open,High,Low,Adj Close,Volume,capmrktcurusd,asseteodcompletiontime,PriceUSD,Close
2989,23/11/2022,16195.58887,16638.19336,16170.50293,16610.70703,32958875628,5.58E+11,1669268890,16522.38878,16610.70703
2990,24/11/2022,16611.63672,16771.47461,16501.76758,16604.46484,26129037414,6.03E+11,1669353221,16522.38878,16604.46484
2991,25/11/2022,16602.26953,16603.31641,16388.4043,16521.8418,18678255976,6.06E+11,1669443392,16522.38878,16521.8418
2992,26/11/2022,null,null,null,null,null,6.27E+11,,16522.38878,null
2993,27/11/2022,16461.58789,16579.22461,16459.1543,16545.61719,19394045952,,,,16545.61719


In [ ]:
df = df.drop(['Column3', 'Column4'], axis=1)


In [12]:
df = df.drop(['Adj Close', 'Volume','capmrktcurusd', 'asseteodcompletiontime'], axis=1)
df.head()


,Date,Open,High,Low,PriceUSD,Close
0,17/09/2014,465.864014,468.174011,452.421997,267.7976174,457.334015
1,18/09/2014,456.859985,456.859985,413.104004,259.6136959,424.440002
2,19/09/2014,424.102997,427.834991,384.532013,236.3808189,394.79599
3,20/09/2014,394.673004,423.29599,389.882996,217.1934725,408.903992
4,21/09/2014,408.084991,412.425995,393.181,234.3929975,398.821014


In [13]:
# Convert the date column to datetime format
df['Date'] = pd.to_datetime(df['Date'])

In [14]:
df.head()

,Date,Open,High,Low,PriceUSD,Close
0,2014-09-17,465.864014,468.174011,452.421997,267.7976174,457.334015
1,2014-09-18,456.859985,456.859985,413.104004,259.6136959,424.440002
2,2014-09-19,424.102997,427.834991,384.532013,236.3808189,394.79599
3,2014-09-20,394.673004,423.29599,389.882996,217.1934725,408.903992
4,2014-09-21,408.084991,412.425995,393.181,234.3929975,398.821014


In [15]:
# Extract year, month, and day columns from the date column
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month
df['Day'] = df['Date'].dt.day


In [16]:
df = df.drop(['Date'], axis=1)
df.head()


,Open,High,Low,PriceUSD,Close,Year,Month,Day
0,465.864014,468.174011,452.421997,267.7976174,457.334015,2014,9,17
1,456.859985,456.859985,413.104004,259.6136959,424.440002,2014,9,18
2,424.102997,427.834991,384.532013,236.3808189,394.79599,2014,9,19
3,394.673004,423.29599,389.882996,217.1934725,408.903992,2014,9,20
4,408.084991,412.425995,393.181,234.3929975,398.821014,2014,9,21


In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2994 entries, 0 to 2993
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Open      2994 non-null   object
 1   High      2994 non-null   object
 2   Low       2994 non-null   object
 3   PriceUSD  2994 non-null   object
 4   Close     2994 non-null   object
 5   Year      2994 non-null   int64 
 6   Month     2994 non-null   int64 
 7   Day       2994 non-null   int64 
dtypes: int64(3), object(5)
memory usage: 187.2+ KB


In [19]:
df = df.apply(pd.to_numeric, errors='coerce')

print(df.dtypes)

Open        float64
High        float64
Low         float64
PriceUSD    float64
Close       float64
Year          int64
Month         int64
Day           int64
dtype: object


In [20]:
df.head()

,Open,High,Low,PriceUSD,Close,Year,Month,Day
0,465.864014,468.174011,452.421997,267.797617,457.334015,2014,9,17
1,456.859985,456.859985,413.104004,259.613696,424.440002,2014,9,18
2,424.102997,427.834991,384.532013,236.380819,394.795990,2014,9,19
3,394.673004,423.295990,389.882996,217.193473,408.903992,2014,9,20
4,408.084991,412.425995,393.181000,234.392998,398.821014,2014,9,21
